In [1]:
!pip install dash plotly

In [2]:
import pandas as pd
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go

In [6]:

df = pd.read_csv('grouped\grouped_by_subcat_2023.csv', encoding='UTF-8')

app = dash.Dash(__name__)

# Min Max week
min_week = df['Week number'].min()
max_week = df['Week number'].max()

# 
app.layout = html.Div([
    html.H1("Dashboard"),
    html.Label("Select the categories"),
    dcc.Dropdown(
        id='category-dropdown',
        options=[{'label': cat, 'value': cat} for cat in df['CATEGORY'].unique()],
        value=[df['CATEGORY'].unique()[0]],
        multi=True  
    ),
    html.Label("Selecet the subcategories"),
    dcc.Dropdown(
        id='subcategory-dropdown',
        multi=True 
    ),
    dcc.Checklist(
        id='group-by-category',
        options=[{'label': 'Group by caterogy', 'value': 'group'}],
        value=[]
    ),
    html.Label("Select week range"),
    dcc.RangeSlider(
        id='week-range-slider',
        min=min_week,
        max=max_week,
        step=1,
        value=[min_week, max_week],
        marks={i: str(i) for i in range(min_week, max_week+1)}
    ),
    dcc.Graph(id='weekly-graph'),
    dcc.Graph(id='bonus-graph')
])

# Subcat update based on selected categories
@app.callback(
    Output('subcategory-dropdown', 'options'),
    Output('subcategory-dropdown', 'value'),
    Input('category-dropdown', 'value')
)
def update_subcategories(selected_categories):
    filtered_df = df[df['CATEGORY'].isin(selected_categories)]
    subcategories = filtered_df['SUBCATEGORY'].unique()
    return [{'label': sub, 'value': sub} for sub in subcategories], list(subcategories)

@app.callback(
    Output('weekly-graph', 'figure'),
    Output('bonus-graph', 'figure'),
    Input('category-dropdown', 'value'),
    Input('subcategory-dropdown', 'value'),
    Input('group-by-category', 'value'),
    Input('week-range-slider', 'value')
)
def update_graphs(selected_categories, selected_subcategories, group_by_category, selected_weeks):
    if 'group' in group_by_category:
        filtered_df = df[
            (df['CATEGORY'].isin(selected_categories)) & 
            (df['Week number'] >= selected_weeks[0]) & 
            (df['Week number'] <= selected_weeks[1])
        ]
        
        # Data groupby
        grouped_data = filtered_df.groupby(['Week number', 'CATEGORY']).sum().reset_index()
        line_fig = px.line(grouped_data, x='Week number', y='QUANTITY', color='CATEGORY', title='Продажи по неделям')
        
        grouped_bonus_data = filtered_df.groupby(['Week number', 'CATEGORY']).sum().reset_index()
        bonus_fig = go.Figure()
        for category in grouped_bonus_data['CATEGORY'].unique():
            category_data = grouped_bonus_data[grouped_bonus_data['CATEGORY'] == category]
            bonus_fig.add_trace(go.Bar(
                x=category_data['Week number'],
                y=category_data['BONUSES_SPENT'],
                name=f'{category} - BONUSES_SPENT'
            ))
            
            bonus_fig.add_trace(go.Bar(
                x=category_data['Week number'],
                y=category_data['BONUSES_RECIVE'],
                name=f'{category} - BONUSES_RECIVE'
            ))
            
            bonus_fig.add_trace(go.Scatter(
                x=category_data['Week number'],
                y=category_data['COST (without bonuses)'],
                mode='lines+markers',
                name=f'{category} - COST (without bonuses)',
                yaxis='y2'
            ))
            
        bonus_fig.update_layout(
            barmode='group',
            title='Bonuses and cost (without bonuses) per week',
            xaxis_title='Week number',
            yaxis_title='Bonuses',
            yaxis2=dict(
                title='Cost (without bonuses)',
                overlaying='y',
                side='right'
            ),
            legend_title='Category'
        )
    else:
        if not selected_categories:
            return {}, {}
        filtered_df = df[
            (df['CATEGORY'].isin(selected_categories)) & 
            (df['SUBCATEGORY'].isin(selected_subcategories)) & 
            (df['Week number'] >= selected_weeks[0]) & 
            (df['Week number'] <= selected_weeks[1])
        ]
        
        weekly_data = filtered_df.groupby(['Week number', 'CATEGORY', 'SUBCATEGORY']).sum().reset_index()
        line_fig = px.line(weekly_data, x='Week number', y='QUANTITY', color='SUBCATEGORY', title='Sales per week')
        
        bonus_data = filtered_df.groupby(['Week number', 'CATEGORY', 'SUBCATEGORY']).sum().reset_index()
        bonus_fig = go.Figure()
        for subcategory in bonus_data['SUBCATEGORY'].unique():
            subcategory_data = bonus_data[bonus_data['SUBCATEGORY'] == subcategory]
            bonus_fig.add_trace(go.Bar(
                x=subcategory_data['Week number'],
                y=subcategory_data['BONUSES_SPENT'],
                name=f'{subcategory} - BONUSES_SPENT'
            ))
            
            bonus_fig.add_trace(go.Bar(
                x=subcategory_data['Week number'],
                y=subcategory_data['BONUSES_RECIVE'],
                name=f'{subcategory} - BONUSES_RECIVE'
            ))
            
            bonus_fig.add_trace(go.Scatter(
                x=subcategory_data['Week number'],
                y=subcategory_data['COST (without bonuses)'],
                mode='lines+markers',
                name=f'{subcategory} - COST (without bonuses)',
                yaxis='y2'
            ))

    bonus_fig.update_layout(
        barmode='group',
        title='Bonuses and cost (without bonuses) per week',
        xaxis_title='Week number',
        yaxis_title='Bonuses',
        yaxis2=dict(
            title='Cost (without bonuses)',
            overlaying='y',
            side='right'
        ),
        legend_title='Subcategory'
    )

    return line_fig, bonus_fig

app.run_server(debug=True)